In [163]:
from keras.models import Sequential,Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector,Merge,BatchNormalization,Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Embedding,LSTM,GRU,TimeDistributed,RepeatVector,Merge,Input,merge,UpSampling2D
from keras.preprocessing import sequence
from keras import callbacks
from keras.optimizers import SGD, RMSprop, Adam

import numpy as np
from vgg16 import Vgg16
import matplotlib.pyplot as plt
import PIL.Image

from tqdm import tqdm

import utils as utils

import cPickle as pickle
import string

import collections
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

import re
from numpy.random import random, permutation, randn, normal 

import os

import preprocessing as preproc

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from matplotlib import animation
from IPython.display import display, HTML

import pandas as pd

import nnModel as nnModel


In [32]:
import skvideo.io

In [33]:
vc = skvideo.io.VideoCapture()

In [156]:
def plot_movie_mp4(image_array):
    dpi = 72.0
    xpixels, ypixels = image_array[0].shape[0], image_array[0].shape[1]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        im.set_array(image_array[i])
        return (im,)

    anim = animation.FuncAnimation(fig, animate, frames=len(image_array), interval = 50)
    display(HTML(anim.to_html5_video()))

def resize(pic):
    return np.asarray(Image.fromarray(np.uint8(pic)).resize((224, 224), Image.NEAREST))

def read_video_frames(filename):
    vid = imageio.get_reader(filename,  'ffmpeg')
    
    frames = []
    
    for i, im in enumerate(vid):
        im = np.asarray(im)
        im = np.expand_dims(im,axis=0)
        frames.append(im)
    
    frames = np.vstack(frames)
    
    return [resize(pic) for pic in frames]

def get_mp4_vid_frames(path):
    
    frames = read_video_frames(filename)

    frames = np.transpose(np.asarray(frames),(0,3,1,2))
        
    return frames

## Load video

In [160]:
horse_path = "./horse.mp4"
video_frames = get_mp4_vid_frames(horse_path)

In [161]:
video_frames.shape

(287, 3, 224, 224)

## Load model

In [196]:
base_path = data_path+"app-100-length-15/"

(unique_words, word2index, index2word) = utils.load_language_data_structures(base_path + general_datastruct_folder)
VOCAB_SIZE = len(unique_words)
MAX_CAPTION_LEN = 15
EMB_SIZE = 300

In [197]:
emb = nnModel.get_embeddings(index2word, VOCAB_SIZE, EMB_SIZE)
model = nnModel.build_model(emb,MAX_CAPTION_LEN, VOCAB_SIZE, EMB_SIZE)

model.load_weights(save_path + models_folder+"big/" +'app_100_length_15_past_word_20_epoch_300d_gru_2x1048_big.h5')

Found = 840
Not found = 8


In [241]:
(_,video_captions) = nnModel.make_prediction_on_dataset(video_frames,model,word2index,index2word,MAX_CAPTION_LEN,0,100)
video_captions = list(set(video_captions))

100%|██████████| 100/100 [00:11<00:00,  9.93it/s]


In [242]:
video_captions

[u'A horse is grazing in a field with a fence behind it',
 u'A horse standing in a field with other horses',
 u'A horse standing in a field with a fence behind it',
 u'A horse standing in a field with a horse behind it',
 u'A horse standing in a field with a fence in the background',
 u'A horse is grazing in a field of grass']

In [233]:
import videoExplorer as vidExplorer

## Search frames by word

In [237]:
reload(vidExplorer)

<module 'videoExplorer' from 'videoExplorer.py'>

In [243]:
video_search_word = "fence"
(found_images,found_captions,found_indexes) = vidExplorer.search_video_by(video_search_word,video_frames,video_captions)